# Here we use AdEx, spiking neuron model for modeling hippocampal Gamma Oscillations

In [6]:
# inintializing parameters
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [7]:
tmax=120000                        # max time of simulation
dt=0.05                            # sampling time (ms)
fs=1/(dt/1000)                     #sampling frequency in HZ
num_rep=6                          # number of repetition
F=np.arange(10,50,1)               # frequency vector for spectrogram
window=0.2*fs;                     # time window of spectrogram
novelap=0.9*window                 # overlap in moving window for spectrogram (90% overlap)
astim=30                           # stimulation amplitude
fstim=5                            # stimulation frequency
simtype="AM"                       # stimulation type (ac,dc,AM)
tstart=2*fs                        # stimulation start point (ms)
tstop=4*fs                         # stimulation end time (ms)
timev=np.arange(0,tmax-1,1/fs)     # time vector
f_carier=1000

Initialize number of neurons and single cell properties
Time constant of excitatory neurons: 10 mS, inhibitory: 10 mS
The network consist of 1000 adaptive-exponential integrate and fire model (AdEx) with 80% excitatory and 20% inhibitory

In [64]:
N=1000                                         # total number of neurons in the active network
Ne=int(0.8*N)                                  # number of excitatory neurons in network
Ni_GABAa=int(0.2*N)                            # number of inhibitory neurons (GABAa)
Ni_GABAb=int(0.05*N)                           # number of inhibitory neurons (GABAb type- slow)
Vr=np.zeros((Ne+Ni_GABAa+Ni_GABAb,1))          # reset voltage for all the neurons
Vr[0:Ne]=-55*np.ones((Ne,1))
Vr[Ne:Ne+Ni_GABAa]=-62*np.ones((Ni_GABAa,1))
Vr[Ne+Ni_GABAa:Ne+Ni_GABAa+Ni_GABAb]=-67*np.ones((Ni_GABAb,1))
#Vr=np.array(-55*np.ones((Ne,1)),-62*np.ones((Ni_GABAa,1)),-67*np.ones((Ni_GABAb,1)))

gL= np.zeros((Ne+Ni_GABAa+Ni_GABAb,1))         #leake conductance (nS)
gL[0:Ne]=100*np.ones((Ne,1))
gL[Ne:Ne+Ni_GABAa]=-100*np.ones((Ni_GABAa,1))
gL[Ne+Ni_GABAa:Ne+Ni_GABAa+Ni_GABAb]=100*np.ones((Ni_GABAb,1))  

EL=np.zeros((Ne+Ni_GABAa+Ni_GABAb,1))         # mV
EL[0:Ne]=-55*np.ones((Ne,1))
EL[Ne:Ne+Ni_GABAa]=-62*np.ones((Ni_GABAa,1))
EL[Ne+Ni_GABAa:Ne+Ni_GABAa+Ni_GABAb]=-67*np.ones((Ni_GABAb,1))

Cm=50*np.ones((Ne+Ni_GABAa+Ni_GABAb,1))  # membrane capacitance (pF)

VT=np.zeros((Ne+Ni_GABAa+Ni_GABAb,1))         # threshold voltage (mV)
VT[0:Ne]=-55*np.ones((Ne,1))
VT[Ne:Ne+Ni_GABAa]=-62*np.ones((Ni_GABAa,1))
VT[Ne+Ni_GABAa:Ne+Ni_GABAa+Ni_GABAb]=-67*np.ones((Ni_GABAb,1))

tau_w=400*np.ones((Ne+Ni_GABAa+Ni_GABAb,1))
ka= np.zeros((Ne+Ni_GABAa+Ni_GABAb,1))         # threshold voltage (mV)
ka[0:Ne]=2.7*np.ones((Ne,1))
ka[Ne:Ne+Ni_GABAa]=1*np.ones((Ni_GABAa,1))
ka[Ne+Ni_GABAa:Ne+Ni_GABAa+Ni_GABAb]=1*np.ones((Ni_GABAb,1))

b=np.zeros((Ne+Ni_GABAa+Ni_GABAb,1))    #adapting parameters pA
b[0:Ne]=8*np.ones((Ne,1))

a=np.zeros((Ne+Ni_GABAa+Ni_GABAb,1))    # nS
a[0:Ne]=2*np.ones((Ne,1))


In [55]:
## synapce parameters
Vthre=-50                   # (mV) spiking threshold
Ee=0                        # reversal potential excitatory (mV)
Ei=-75                      # reversal potential inhibitory (mV)
noiseNe=500
noiseNi_GABAa=0                #interneouron noise
noiseNi_GABAb=0

Iexc=0
Iinh_a=0
Iinh_b=0
tau_pyr=5
tau_inh_a=8
tau_inh_b=50
W=np.zeros((Ne+Ni_GABAa+Ni_GABAb,Ne+Ni_GABAa+Ni_GABAb))   # synaptic connection matrix 
density=0.4
## synaptic connection matrix

from scipy.sparse import random
s1=random(Ne,Ne,density=0.15)                 # excitatory neourons are connected in sparse way with 0.15 density
W[0:Ne,0:Ne]=0.3*(s1.A)

s2=random(0:Ne,Ne:Ne+Ni_GABAa,density=0.4) 
W[0:Ne,Ne:Ne+Ni_GABAa]=1.4*(s2.A)

s3=random(Ne:Ne+Ni_GABAa,0:Ne,density=0.4) 
W[Ne:Ne+Ni_GABAa,0:Ne]=1.4*(s3.A)

s4=random(0:Ne,0:Ni_GABAb,density=0.4) 
W[0:Ne,Ne+Ni_GABAa:Ne+Ni_GABAa+Ni_GABAb]=0*(s4.A)

s5=random(0:Ni_GABAb,0:Ne,density=0.4) 
W[Ne+Ni_GABAa:Ne+Ni_GABAa+Ni_GABAb,]=0*(s5.A)



In [58]:
import math as m
for jj=1:num_rep:
        v=[]
        v[:,1]=Vr
        Iw=[]
        Istim=np.zeros((Ne+Ni_GABAa+Ni_GABAb,tmax))
        Isyn=np.zeros((Ne+Ni_GABAa+Ni_GABAb,tmax))
        Iw(:,1)=np.zeros((Ne+Ni_GABAa+Ni_GABAb,1))
        firing=[]
        firing_time=np.zeros((Ne+Ni_GABAa+Ni_GABAb,tmax))
        for t=1:tmax:
                if t>tstart and t<tstop:
                    if stimtype=='ac':
                        Istim[0:Ne,t]=astim*np.sin(2*pi*fstim*((t-tstart)/fs))
                    elif stimtype=='dc':
                        Istim[0:Ne,t]=astim
                    elif stimtype=='AM':
                        Istim[0:Ne,t]=0.5*stimAM*(np.sin(2*m.pi*(fstim+f_carier)*((t-tstart)/fs))-np.sin(2*m.pif_carrier*((t-tstart)/fs)))
                        
                
                        
                        
                        
                    
                    
                    
                

array([[0.37828676, 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])